In [1]:
# Improt packages
import pandas as pd
import pyarrow

# Import functions from apps folder
from apps.collect_data import *

In [2]:
# URL of the Wikipedia page
url = "https://en.wikipedia.org/wiki/List_of_Women's_Prize_for_Fiction_winners"

books = get_wikidata(url)
books.head()


,Year,Author,Title,Result
0,2020,Maggie O'Farrell,Hamnet,Winner
1,2020,Angie Cruz,Dominicana,Finalist
2,2020,Bernardine Evaristo,"Girl, Woman, Other",Finalist
3,2020,Natalie Haynes,A Thousand Ships,Finalist
4,2020,Hilary Mantel,The Mirror and the Light,Finalist


In [3]:
books.to_parquet('assets/WPH.parquet')

In [4]:
df = book_info_add(books)

In [5]:
df.to_parquet('assets/GBA.parquet')

In [6]:
# Merge dataframes 
book_df = pd.merge(df,
                     books,
                     on='Title', how='outer')

In [7]:
book_df.to_parquet('assets/book_df.parquet')

In [8]:
book_df.loc[book_df['Author(s)'].isnull()==True]


,Title,Author(s),Publish Date,Description,ISBN,Page Count,Categories,Average Rating,Rating Count,Language,Year,Author,Result


In [9]:
book_df

,Title,Author(s),Publish Date,Description,ISBN,Page Count,Categories,Average Rating,Rating Count,Language,Year,Author,Result
0,Hamnet,Maggie O'Farrell,2020-07-21,NATIONAL BOOK CRITICS CIRCLE AWARD WINNER • NE...,9780525657613,320.0,Fiction,4.0,98.0,en,2020,Maggie O'Farrell,Winner
1,Dominicana,Angie Cruz,2019-09-03,A GOOD MORNING AMERICA BOOK CLUB PICK Shortlis...,9781250205926,381.0,Fiction,4.0,18.0,en,2020,Angie Cruz,Finalist
2,"Girl, Woman, Other",Bernardine Evaristo,2019-11-05,NATIONAL BESTSELLER WINNER OF THE BOOKER PRIZE...,9780802156990,464.0,Fiction,4.5,52.0,en,2020,Bernardine Evaristo,Finalist
3,A Thousand Ships,Natalie Haynes,2019-05-02,Shortlisted for the Women's Prize for Fiction ...,9781509836222,352.0,Fiction,4.0,15.0,en,2020,Natalie Haynes,Finalist
4,The Mirror and the Light,Hilary Mantel,2022-03-29,AN INTERNATIONAL BESTSELLER WINNER OF THE WALT...,1443413747,912.0,nan,4.5,42.0,en,2020,Hilary Mantel,Finalist
...,...,...,...,...,...,...,...,...,...,...,...,...,...
164,The Leper's Companions,Julia Blackburn,2013-10-31,"To escape from her own sadness, a woman finds ...",9781448190904,224.0,Fiction,3.0,1.0,en,1999,Julia Blackburn,Shortlist
165,Visible Worlds,Marilyn Bowering,1999,Two brothers in 1930s Manitoba find themselves...,0006551130,308.0,Arctic regions,NaN,NaN,en,1999,Marilyn Bowering,Shortlist
166,The Short History of a Prince,Jane Hamilton,2010-09-01,Walter McCloud is a boy with dreams unlike mos...,9780307764072,368.0,Fiction,NaN,NaN,en,1999,Jane Hamilton,Shortlist
167,The Poisonwood Bible,Barbara Kingsolver,2008-09-04,**NOW INCLUDING THE FIRST CHAPTER OF DEMON COP...,9780571246212,633.0,Fiction,4.0,285.0,en,1999,Barbara Kingsolver,Shortlist


In [35]:


# def get_book_topics(title, author):
#     import requests
#     import pandas as pd
#     base_url = 'http://openlibrary.org/search.json'
#     params = {
#         'title': title,
#         'author': author,
#         'limit': 1
#     }

#     response = requests.get(base_url, params=params)
#     data = response.json()

#     if 'docs' in data and len(data['docs']) > 0:
#         book = data['docs'][0]
#         topics = book.get('subject', [])
#         return topics
#     else:
#         return []

# Collect topics for each book
topics_list = {}
for _, row in book_df.head(3).iterrows():
    title = row['Title']
    author = row['Author']
    topics = get_book_topics(title, author)
    # creates a dictionary with the title as key and a list of the topics
    book_dict = {row['Title']: list(topics)}
    # creates full dict of all titles
    topics_list.update(book_dict)

# dict of topics 
topics_list

{'Hamnet': ['English literature',
  'Fiction, historical',
  'Fiction, biographical',
  'fiction',
  'nyt:hardcover-fiction=2020-12-27',
  'New York Times bestseller',
  'New York Times reviewed',
  'Fiction',
  'FICTION / Historical / General'],
 'Dominicana': ['Fiction, coming of age',
  'New york (n.y.), fiction',
  'Hispanic americans, fiction',
  'Fiction, family life',
  'Marriage, fiction',
  'New York Times reviewed',
  'Fiction, family life, general',
  'Immigrants',
  'Fiction',
  'Women immigrants',
  'Man-woman relationships',
  'Families',
  'Dominicans (Dominican Republic)',
  'Arranged marriage',
  'History',
  'Emigration and immigration',
  'Immigrantes',
  'Romans, nouvelles',
  'Relations entre hommes et femmes',
  'Familles',
  'Dominicains (Peuple)',
  'Mariage arrangé',
  'Histoire',
  'FICTION / Historical / General',
  'FICTION / Hispanic & Latino',
  'FICTION / Coming of Age'],
 'Girl, Woman, Other': ['black women',
  'lgbt',
  'English literature',
  'Fiction,

In [32]:
import json 

with open("assets/topics.json", "w") as outfile:
    json.dump(topics_list, outfile)

In [15]:
def get_book_topics(title, author):
    base_url = 'http://openlibrary.org/search.json'
    params = {
        'title': title,
        'author': author,
        'limit': 1
    }

    response = requests.get(base_url, params=params)
    data = response.json()

    if 'docs' in data and len(data['docs']) > 0:
        book = data['docs'][0]
        topics = book.get('subject', [])
        return topics
    else:
        return []
    
title = "Hamnet"
author = "Maggie O'Farrell"
t = get_book_topics(title, author)
t

['English literature',
 'Fiction, historical',
 'Fiction, biographical',
 'fiction',
 'nyt:hardcover-fiction=2020-12-27',
 'New York Times bestseller',
 'New York Times reviewed',
 'Fiction',
 'FICTION / Historical / General']